# 🚀 Production ML & MLOps: From Notebook to Production

**Phase 7: Превращаем ML-эксперимент в Production-Ready систему**

---

## 🎯 The Production Gap

### Проблема: 90% ML-проектов не доходят до production!

**Почему?**

```python
# Jupyter Notebook (10% работы)
model = RandomForestClassifier()
model.fit(X_train, y_train)
accuracy = model.score(X_test, y_test)
print(f"Accuracy: {accuracy}")
# "Готово! Модель работает!" 🎉
```

**Но для production нужно (90% работы):**

- 📦 **Packaging:** Как упаковать модель для deployment?
- 🌐 **API:** Как получать predictions по HTTP?
- 🐳 **Containerization:** Как обеспечить reproducibility?
- 📊 **Monitoring:** Как узнать, что модель degraded?
- 🔄 **CI/CD:** Как автоматизировать deployment?
- 📝 **Versioning:** Как отслеживать эксперименты и данные?
- ⚡ **Scaling:** Как обрабатывать 1000+ requests/sec?
- 🔒 **Security:** Authentication, rate limiting, input validation?

---

## 🔧 MLOps = ML + DevOps + Data Engineering

### MLOps Lifecycle:

```
┌─────────────┐    ┌─────────────┐    ┌─────────────┐
│   DATA      │ → │   MODEL     │ → │   DEPLOY    │
│  PIPELINE   │    │  TRAINING   │    │   & SERVE   │
└─────────────┘    └─────────────┘    └─────────────┘
       ↑                                      │
       │         ┌─────────────┐              │
       └──────── │  MONITOR &  │ ←───────────┘
                 │   RETRAIN   │
                 └─────────────┘
```

### Ключевые компоненты:

#### 1. **Experiment Tracking**
- Логирование параметров, метрик, артефактов
- Сравнение экспериментов
- Reproducibility
- **Tools:** MLflow, Weights & Biases, Neptune, TensorBoard

#### 2. **Model Registry**
- Версионирование моделей
- Staging → Production переходы
- Model lineage
- **Tools:** MLflow Model Registry, SageMaker Model Registry

#### 3. **Data Versioning**
- Version control для данных
- Reproducibility экспериментов
- Data lineage
- **Tools:** DVC, Delta Lake, LakeFS

#### 4. **Feature Store**
- Централизованное хранилище features
- Online/offline serving
- Feature sharing между моделями
- **Tools:** Feast, Tecton, AWS Feature Store

#### 5. **Model Serving**
- REST API для predictions
- Batch vs Real-time inference
- A/B testing
- **Tools:** FastAPI, BentoML, Seldon, KServe

#### 6. **Monitoring**
- Data drift detection
- Model performance monitoring
- Alerting
- **Tools:** Evidently, WhyLabs, Arize, Prometheus/Grafana

#### 7. **CI/CD for ML**
- Automated testing (data, model, API)
- Automated deployment
- **Tools:** GitHub Actions, Jenkins, GitLab CI, Kubeflow Pipelines

---

## 📊 Что мы реализуем

### Full Production Pipeline:

**Part 1: Experiment Tracking**
1. MLflow setup и logging
2. Tracking parameters, metrics, models
3. Model comparison и selection
4. Model Registry

**Part 2: Model Serialization**
1. Pickle, joblib, ONNX
2. Custom transformers
3. Pipeline serialization

**Part 3: API Development (FastAPI)**
1. REST API для ML модели
2. Pydantic schemas для validation
3. Async endpoints
4. Error handling
5. API testing

**Part 4: Containerization (Docker)**
1. Dockerfile для ML приложения
2. Multi-stage builds
3. Docker Compose
4. Best practices для ML containers

**Part 5: Monitoring & Drift Detection**
1. Data drift (input distribution changes)
2. Concept drift (target relationship changes)
3. Model performance monitoring
4. Alerting strategies

**Part 6: CI/CD для ML**
1. GitHub Actions workflow
2. Automated testing
3. Automated deployment
4. Continuous training

**Part 7: Data Versioning (DVC)**
1. DVC basics
2. Tracking data и models
3. Pipelines

---


## 💻 Часть 1: Setup и Experiment Tracking

### 1.1 Импорт библиотек

In [ ]:
# Базовые библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Sklearn
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, classification_report
)

# MLOps Tools
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient

# Serialization
import pickle
import joblib
import json
import os
from datetime import datetime

# Type hints
from typing import Dict, List, Any, Tuple, Optional

# Настройки
plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)

print("\n✅ Все библиотеки загружены")
print(f"MLflow version: {mlflow.__version__}")


### 1.2 Dataset для демонстрации

Используем синтетический dataset для **Customer Churn Prediction** - типичная production задача.

In [ ]:
def create_churn_dataset(n_samples: int = 10000) -> pd.DataFrame:
    """
    Create synthetic customer churn dataset.
    
    Features:
    - tenure: months with company
    - monthly_charges: monthly bill
    - total_charges: total amount paid
    - contract_type: month-to-month, one year, two year
    - payment_method: credit card, bank transfer, electronic check
    - internet_service: DSL, Fiber optic, No
    - online_security: Yes, No
    - tech_support: Yes, No
    - num_tickets: support tickets opened
    - senior_citizen: 0 or 1
    
    Target: churn (0 or 1)
    """
    np.random.seed(42)
    
    # Numerical features
    tenure = np.random.randint(1, 72, n_samples)
    monthly_charges = np.random.uniform(20, 100, n_samples)
    total_charges = tenure * monthly_charges + np.random.normal(0, 100, n_samples)
    total_charges = np.maximum(total_charges, 0)
    num_tickets = np.random.poisson(2, n_samples)
    
    # Categorical features
    contract_type = np.random.choice(
        ['Month-to-month', 'One year', 'Two year'],
        n_samples, p=[0.5, 0.3, 0.2]
    )
    payment_method = np.random.choice(
        ['Credit card', 'Bank transfer', 'Electronic check'],
        n_samples, p=[0.4, 0.3, 0.3]
    )
    internet_service = np.random.choice(
        ['DSL', 'Fiber optic', 'No'],
        n_samples, p=[0.4, 0.4, 0.2]
    )
    online_security = np.random.choice(['Yes', 'No'], n_samples, p=[0.4, 0.6])
    tech_support = np.random.choice(['Yes', 'No'], n_samples, p=[0.3, 0.7])
    senior_citizen = np.random.choice([0, 1], n_samples, p=[0.84, 0.16])
    
    # Create churn with realistic patterns
    churn_prob = (
        0.1 +  # baseline
        (tenure < 12).astype(float) * 0.15 +  # short tenure = higher churn
        (monthly_charges > 70).astype(float) * 0.1 +  # high charges = higher churn
        (contract_type == 'Month-to-month').astype(float) * 0.2 +  # month-to-month = higher churn
        (payment_method == 'Electronic check').astype(float) * 0.05 +  # electronic check = higher churn
        (online_security == 'No').astype(float) * 0.05 +  # no security = higher churn
        (tech_support == 'No').astype(float) * 0.05 +  # no support = higher churn
        (num_tickets > 3).astype(float) * 0.1  # many tickets = higher churn
    )
    
    # Normalize probabilities
    churn_prob = np.clip(churn_prob, 0, 0.9)
    churn = (np.random.random(n_samples) < churn_prob).astype(int)
    
    df = pd.DataFrame({
        'tenure': tenure,
        'monthly_charges': monthly_charges,
        'total_charges': total_charges,
        'num_tickets': num_tickets,
        'contract_type': contract_type,
        'payment_method': payment_method,
        'internet_service': internet_service,
        'online_security': online_security,
        'tech_support': tech_support,
        'senior_citizen': senior_citizen,
        'churn': churn
    })
    
    return df

# Создаём dataset
df = create_churn_dataset(10000)

print("\n📊 Customer Churn Dataset")
print("=" * 50)
print(f"Shape: {df.shape}")
print(f"Churn rate: {df['churn'].mean():.2%}")
print(f"\nFeature types:")
print(df.dtypes)
print(f"\nFirst rows:")
df.head()


In [ ]:
# Preprocessing
print("\n" + "=" * 50)
print("PREPROCESSING")
print("=" * 50)

# Определяем типы признаков
numerical_cols = ['tenure', 'monthly_charges', 'total_charges', 'num_tickets']
categorical_cols = [
    'contract_type', 'payment_method', 'internet_service',
    'online_security', 'tech_support', 'senior_citizen'
]

# senior_citizen уже числовой, но для демонстрации обрабатываем как категориальный
df['senior_citizen'] = df['senior_citizen'].astype(str)

# Label encoding для categorical
label_encoders = {}
df_processed = df.copy()

for col in categorical_cols:
    le = LabelEncoder()
    df_processed[col] = le.fit_transform(df_processed[col])
    label_encoders[col] = le

# Feature matrix и target
feature_cols = numerical_cols + categorical_cols
X = df_processed[feature_cols]
y = df_processed['churn']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Scaling
scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()
X_train_scaled[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test_scaled[numerical_cols] = scaler.transform(X_test[numerical_cols])

print(f"\n✅ Preprocessing completed")
print(f"Train size: {X_train.shape[0]}")
print(f"Test size: {X_test.shape[0]}")
print(f"Features: {len(feature_cols)}")
print(f"Train churn rate: {y_train.mean():.2%}")
print(f"Test churn rate: {y_test.mean():.2%}")


---

## 🔬 Часть 2: Experiment Tracking с MLflow

### Что такое MLflow?

**MLflow** - open-source платформа для управления ML lifecycle:

1. **MLflow Tracking:** Логирование экспериментов
2. **MLflow Projects:** Packaging code для reproducibility
3. **MLflow Models:** Packaging models для deployment
4. **MLflow Model Registry:** Версионирование и staging моделей

### Почему Experiment Tracking важен?

**Без tracking:**
```python
# model_v1.py, model_v2_final.py, model_v2_final_FINAL.py...
# "Какие параметры дали лучший результат?"
# "Какую версию мы задеплоили?"
# "Почему accuracy упала?"
```

**С MLflow:**
- ✅ Все эксперименты логируются автоматически
- ✅ Параметры, метрики, артефакты сохранены
- ✅ Сравнение экспериментов в UI
- ✅ Reproducibility гарантирован
- ✅ Model lineage отслеживается

---


In [ ]:
print("=" * 70)
print("MLFLOW EXPERIMENT TRACKING")
print("=" * 70)

# Создаём эксперимент
experiment_name = "churn_prediction"
mlflow.set_experiment(experiment_name)

print(f"\n✅ MLflow experiment created: {experiment_name}")
print(f"Tracking URI: {mlflow.get_tracking_uri()}")

# Получаем experiment info
experiment = mlflow.get_experiment_by_name(experiment_name)
print(f"Experiment ID: {experiment.experiment_id}")
print(f"Artifact Location: {experiment.artifact_location}")


### 2.1 Логирование экспериментов

Обучим несколько моделей и залогируем все детали в MLflow.

In [ ]:
def train_and_log_model(
    model,
    model_name: str,
    X_train: pd.DataFrame,
    X_test: pd.DataFrame,
    y_train: pd.Series,
    y_test: pd.Series,
    params: Dict[str, Any]
) -> str:
    """
    Train model and log everything to MLflow.
    
    Returns:
        run_id: MLflow run ID
    """
    with mlflow.start_run(run_name=model_name):
        # Log parameters
        mlflow.log_params(params)
        mlflow.log_param("model_type", model_name)
        mlflow.log_param("train_size", len(X_train))
        mlflow.log_param("test_size", len(X_test))
        mlflow.log_param("n_features", X_train.shape[1])
        
        # Train model
        model.fit(X_train, y_train)
        
        # Predictions
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        
        # Calculate metrics
        metrics = {
            "accuracy": accuracy_score(y_test, y_pred),
            "precision": precision_score(y_test, y_pred),
            "recall": recall_score(y_test, y_pred),
            "f1": f1_score(y_test, y_pred),
            "roc_auc": roc_auc_score(y_test, y_pred_proba)
        }
        
        # Log metrics
        mlflow.log_metrics(metrics)
        
        # Log model
        mlflow.sklearn.log_model(
            model,
            artifact_path="model",
            registered_model_name=f"{model_name}_churn"
        )
        
        # Log additional artifacts
        # Save feature importance (for tree-based models)
        if hasattr(model, 'feature_importances_'):
            importance_df = pd.DataFrame({
                'feature': feature_cols,
                'importance': model.feature_importances_
            }).sort_values('importance', ascending=False)
            
            importance_path = f"/tmp/{model_name}_importance.csv"
            importance_df.to_csv(importance_path, index=False)
            mlflow.log_artifact(importance_path)
        
        # Log confusion matrix as artifact
        from sklearn.metrics import confusion_matrix
        cm = confusion_matrix(y_test, y_pred)
        cm_df = pd.DataFrame(
            cm,
            columns=['Predicted 0', 'Predicted 1'],
            index=['Actual 0', 'Actual 1']
        )
        cm_path = f"/tmp/{model_name}_confusion_matrix.csv"
        cm_df.to_csv(cm_path)
        mlflow.log_artifact(cm_path)
        
        # Get run ID
        run_id = mlflow.active_run().info.run_id
        
        print(f"\n✅ {model_name} logged to MLflow")
        print(f"   Run ID: {run_id}")
        print(f"   Metrics:")
        for metric_name, value in metrics.items():
            print(f"      {metric_name}: {value:.4f}")
        
        return run_id


In [ ]:
# Обучаем несколько моделей
print("\n" + "=" * 70)
print("TRAINING MODELS WITH MLFLOW TRACKING")
print("=" * 70)

run_ids = {}

# 1. Logistic Regression
print("\n[1/4] Logistic Regression")
lr_params = {
    "C": 1.0,
    "max_iter": 1000,
    "solver": "lbfgs"
}
lr = LogisticRegression(**lr_params, random_state=42)
run_ids['LogisticRegression'] = train_and_log_model(
    lr, "LogisticRegression",
    X_train_scaled, X_test_scaled, y_train, y_test,
    lr_params
)

# 2. Random Forest (default)
print("\n[2/4] Random Forest (default)")
rf_params = {
    "n_estimators": 100,
    "max_depth": 10,
    "min_samples_split": 10
}
rf = RandomForestClassifier(**rf_params, random_state=42, n_jobs=-1)
run_ids['RandomForest_v1'] = train_and_log_model(
    rf, "RandomForest_v1",
    X_train_scaled, X_test_scaled, y_train, y_test,
    rf_params
)

# 3. Random Forest (tuned)
print("\n[3/4] Random Forest (tuned)")
rf_params_v2 = {
    "n_estimators": 200,
    "max_depth": 15,
    "min_samples_split": 5,
    "min_samples_leaf": 2
}
rf_v2 = RandomForestClassifier(**rf_params_v2, random_state=42, n_jobs=-1)
run_ids['RandomForest_v2'] = train_and_log_model(
    rf_v2, "RandomForest_v2",
    X_train_scaled, X_test_scaled, y_train, y_test,
    rf_params_v2
)

# 4. Gradient Boosting
print("\n[4/4] Gradient Boosting")
gb_params = {
    "n_estimators": 100,
    "max_depth": 5,
    "learning_rate": 0.1
}
gb = GradientBoostingClassifier(**gb_params, random_state=42)
run_ids['GradientBoosting'] = train_and_log_model(
    gb, "GradientBoosting",
    X_train_scaled, X_test_scaled, y_train, y_test,
    gb_params
)

print("\n" + "=" * 70)
print("✅ ALL MODELS LOGGED TO MLFLOW")
print("=" * 70)
print(f"\nTotal runs: {len(run_ids)}")
print("Run IDs:")
for name, run_id in run_ids.items():
    print(f"  {name}: {run_id}")


### 2.2 Сравнение экспериментов

MLflow позволяет легко сравнивать runs и выбирать лучшую модель.

In [ ]:
# Получаем все runs из эксперимента
client = MlflowClient()

# Search runs
runs = mlflow.search_runs(
    experiment_ids=[experiment.experiment_id],
    order_by=["metrics.roc_auc DESC"]
)

print("\n" + "=" * 70)
print("EXPERIMENT COMPARISON")
print("=" * 70)

# Показываем comparison table
comparison_cols = [
    'run_id', 'tags.mlflow.runName',
    'metrics.accuracy', 'metrics.precision', 'metrics.recall',
    'metrics.f1', 'metrics.roc_auc'
]

comparison_df = runs[comparison_cols].copy()
comparison_df.columns = ['Run ID', 'Model', 'Accuracy', 'Precision', 'Recall', 'F1', 'ROC AUC']
comparison_df['Run ID'] = comparison_df['Run ID'].str[:8] + '...'

print("\nAll Runs (sorted by ROC AUC):")
print(comparison_df.to_string(index=False))

# Best model
best_run = runs.iloc[0]
print(f"\n🏆 Best Model: {best_run['tags.mlflow.runName']}")
print(f"   ROC AUC: {best_run['metrics.roc_auc']:.4f}")
print(f"   F1 Score: {best_run['metrics.f1']:.4f}")
print(f"   Run ID: {best_run['run_id']}")


In [ ]:
# Визуализация сравнения моделей
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Metrics comparison
metrics_to_plot = ['Accuracy', 'Precision', 'Recall', 'F1', 'ROC AUC']
comparison_df.set_index('Model')[metrics_to_plot].plot(
    kind='bar', ax=axes[0], width=0.8
)
axes[0].set_title('Model Performance Comparison', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Score')
axes[0].set_ylim([0, 1])
axes[0].legend(loc='lower right')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(axis='y', alpha=0.3)

# ROC AUC comparison
colors = ['#e74c3c' if i == 0 else '#3498db' for i in range(len(comparison_df))]
axes[1].barh(comparison_df['Model'], comparison_df['ROC AUC'], color=colors)
axes[1].set_title('ROC AUC Comparison', fontsize=14, fontweight='bold')
axes[1].set_xlabel('ROC AUC')
for i, v in enumerate(comparison_df['ROC AUC']):
    axes[1].text(v + 0.005, i, f'{v:.4f}', va='center')
axes[1].set_xlim([0.5, 1.0])
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 MLflow UI доступен по адресу: mlflow ui")
print("Там можно интерактивно сравнивать эксперименты, смотреть графики, артефакты.")


---

## 📦 Часть 3: Model Serialization

### Зачем сериализовать модель?

1. **Persistence:** Сохранить обученную модель для позднего использования
2. **Deployment:** Загрузить модель в production сервис
3. **Sharing:** Передать модель другой команде
4. **Versioning:** Сохранять разные версии

### Форматы сериализации:

| Формат | Pros | Cons | Use Case |
|--------|------|------|----------|
| **Pickle** | Simple, native Python | Security risks, version issues | Quick prototypes |
| **Joblib** | Efficient for numpy arrays | Python-only | Sklearn models |
| **ONNX** | Cross-platform, optimized | Limited model support | Production inference |
| **MLflow** | Full lineage, metadata | Heavier | Full MLOps pipeline |

---


In [ ]:
print("=" * 70)
print("MODEL SERIALIZATION")
print("=" * 70)

# Выберем лучшую модель (или последнюю trained)
best_model = gb  # Gradient Boosting
model_name = "churn_predictor"

# Create directory for models
import os
model_dir = "./models"
os.makedirs(model_dir, exist_ok=True)

# ========================================
# 1. Pickle
# ========================================
print("\n[1/3] Pickle serialization")
pickle_path = f"{model_dir}/{model_name}.pkl"

with open(pickle_path, 'wb') as f:
    pickle.dump(best_model, f)

pickle_size = os.path.getsize(pickle_path) / 1024  # KB
print(f"✅ Saved: {pickle_path}")
print(f"   Size: {pickle_size:.2f} KB")

# Load and verify
with open(pickle_path, 'rb') as f:
    loaded_pickle = pickle.load(f)
    
pickle_pred = loaded_pickle.predict(X_test_scaled.iloc[:5])
print(f"   Verification: {pickle_pred}")

# ========================================
# 2. Joblib (better for sklearn)
# ========================================
print("\n[2/3] Joblib serialization")
joblib_path = f"{model_dir}/{model_name}.joblib"

joblib.dump(best_model, joblib_path)

joblib_size = os.path.getsize(joblib_path) / 1024
print(f"✅ Saved: {joblib_path}")
print(f"   Size: {joblib_size:.2f} KB")

# Load and verify
loaded_joblib = joblib.load(joblib_path)
joblib_pred = loaded_joblib.predict(X_test_scaled.iloc[:5])
print(f"   Verification: {joblib_pred}")

# ========================================
# 3. Save preprocessing artifacts
# ========================================
print("\n[3/3] Saving preprocessing artifacts")

# Save scaler
scaler_path = f"{model_dir}/scaler.joblib"
joblib.dump(scaler, scaler_path)
print(f"✅ Scaler saved: {scaler_path}")

# Save label encoders
encoders_path = f"{model_dir}/label_encoders.joblib"
joblib.dump(label_encoders, encoders_path)
print(f"✅ Label encoders saved: {encoders_path}")

# Save feature columns
config = {
    'numerical_cols': numerical_cols,
    'categorical_cols': categorical_cols,
    'feature_cols': feature_cols
}
config_path = f"{model_dir}/config.json"
with open(config_path, 'w') as f:
    json.dump(config, f, indent=2)
print(f"✅ Config saved: {config_path}")

print("\n" + "=" * 70)
print("✅ ALL ARTIFACTS SAVED")
print("=" * 70)
print(f"\nDirectory: {model_dir}")
for f in os.listdir(model_dir):
    size = os.path.getsize(f"{model_dir}/{f}") / 1024
    print(f"  {f}: {size:.2f} KB")


---

## 🌐 Часть 4: API Development с FastAPI

### Почему FastAPI?

**FastAPI** - современный Python framework для building APIs:

- ✅ **Very fast:** На уровне Node.js и Go (благодаря Starlette и Pydantic)
- ✅ **Type hints:** Automatic validation и documentation
- ✅ **Async support:** Асинхронная обработка запросов
- ✅ **Auto documentation:** Swagger UI и ReDoc из коробки
- ✅ **Easy testing:** Встроенный test client

### Architecture для ML API:

```
┌─────────────┐     ┌─────────────┐     ┌─────────────┐
│   Client    │ → │   FastAPI   │ → │    Model    │
│  (Request)  │     │   Server    │     │  Inference  │
└─────────────┘     └─────────────┘     └─────────────┘
       │                   │                   │
       │                   ↓                   │
       │         ┌─────────────┐              │
       │         │   Pydantic  │              │
       │         │  Validation │              │
       │         └─────────────┘              │
       │                                       │
       └───────────────────────────────────────┘
                    (Response)
```

---


### 4.1 Pydantic Schemas для Validation

**Pydantic** обеспечивает:
- Type validation
- Data parsing
- JSON serialization
- Auto-documentation

In [ ]:
from pydantic import BaseModel, Field, validator
from typing import List, Optional
from enum import Enum

# Enum для categorical fields
class ContractType(str, Enum):
    month_to_month = "Month-to-month"
    one_year = "One year"
    two_year = "Two year"

class PaymentMethod(str, Enum):
    credit_card = "Credit card"
    bank_transfer = "Bank transfer"
    electronic_check = "Electronic check"

class InternetService(str, Enum):
    dsl = "DSL"
    fiber_optic = "Fiber optic"
    no = "No"

class YesNo(str, Enum):
    yes = "Yes"
    no = "No"

# Request schema
class CustomerData(BaseModel):
    """Schema for customer churn prediction request."""
    
    tenure: int = Field(..., ge=0, le=100, description="Months with company")
    monthly_charges: float = Field(..., ge=0, le=500, description="Monthly bill amount")
    total_charges: float = Field(..., ge=0, description="Total amount paid")
    num_tickets: int = Field(..., ge=0, le=50, description="Number of support tickets")
    contract_type: ContractType = Field(..., description="Contract type")
    payment_method: PaymentMethod = Field(..., description="Payment method")
    internet_service: InternetService = Field(..., description="Internet service type")
    online_security: YesNo = Field(..., description="Online security service")
    tech_support: YesNo = Field(..., description="Tech support service")
    senior_citizen: int = Field(..., ge=0, le=1, description="Senior citizen (0 or 1)")
    
    @validator('total_charges')
    def total_charges_must_be_reasonable(cls, v, values):
        if 'tenure' in values and 'monthly_charges' in values:
            expected = values['tenure'] * values['monthly_charges']
            if v < expected * 0.5:
                raise ValueError(f'Total charges too low for tenure and monthly charges')
        return v
    
    class Config:
        schema_extra = {
            "example": {
                "tenure": 24,
                "monthly_charges": 65.5,
                "total_charges": 1500.0,
                "num_tickets": 2,
                "contract_type": "One year",
                "payment_method": "Credit card",
                "internet_service": "Fiber optic",
                "online_security": "Yes",
                "tech_support": "No",
                "senior_citizen": 0
            }
        }

# Response schema
class PredictionResponse(BaseModel):
    """Schema for churn prediction response."""
    
    customer_id: Optional[str] = Field(None, description="Customer ID if provided")
    churn_prediction: int = Field(..., description="Churn prediction (0 or 1)")
    churn_probability: float = Field(..., ge=0, le=1, description="Probability of churn")
    confidence: str = Field(..., description="Confidence level (Low/Medium/High)")
    risk_factors: List[str] = Field(default=[], description="Top risk factors")
    
    class Config:
        schema_extra = {
            "example": {
                "customer_id": "CUST-123",
                "churn_prediction": 1,
                "churn_probability": 0.73,
                "confidence": "High",
                "risk_factors": ["Month-to-month contract", "No tech support"]
            }
        }

# Batch request/response
class BatchPredictionRequest(BaseModel):
    """Schema for batch predictions."""
    customers: List[CustomerData]

class BatchPredictionResponse(BaseModel):
    """Schema for batch prediction response."""
    predictions: List[PredictionResponse]
    total_processed: int
    processing_time_ms: float

# Health check
class HealthResponse(BaseModel):
    """Health check response."""
    status: str
    model_loaded: bool
    model_version: str
    timestamp: str

print("✅ Pydantic schemas defined")
print("\nSchemas:")
print("  - CustomerData (request)")
print("  - PredictionResponse (response)")
print("  - BatchPredictionRequest/Response")
print("  - HealthResponse")


### 4.2 FastAPI Application

Создадим полноценное API для serving ML модели.

In [ ]:
# FastAPI Application Code
# Этот код обычно в отдельном файле app.py

fastapi_code = '''
"""FastAPI application for Churn Prediction ML Model."""

from fastapi import FastAPI, HTTPException, Depends, BackgroundTasks
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
import uvicorn
import joblib
import json
import numpy as np
import pandas as pd
from datetime import datetime
import time
import logging
from typing import List, Optional
from pydantic import BaseModel, Field
from enum import Enum

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize FastAPI app
app = FastAPI(
    title="Churn Prediction API",
    description="ML API for predicting customer churn",
    version="1.0.0",
    docs_url="/docs",  # Swagger UI
    redoc_url="/redoc"  # ReDoc
)

# CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Global variables for model and artifacts
model = None
scaler = None
label_encoders = None
config = None
model_version = "1.0.0"

@app.on_event("startup")
async def load_model():
    """Load model and artifacts on startup."""
    global model, scaler, label_encoders, config
    
    logger.info("Loading model and artifacts...")
    
    try:
        model = joblib.load("models/churn_predictor.joblib")
        scaler = joblib.load("models/scaler.joblib")
        label_encoders = joblib.load("models/label_encoders.joblib")
        
        with open("models/config.json", "r") as f:
            config = json.load(f)
        
        logger.info("✅ Model and artifacts loaded successfully")
    except Exception as e:
        logger.error(f"❌ Failed to load model: {e}")
        raise

def preprocess_input(data: dict) -> np.ndarray:
    """Preprocess input data for prediction."""
    # Create DataFrame
    df = pd.DataFrame([data])
    
    # Convert senior_citizen to string for encoding
    df[\'senior_citizen\'] = df[\'senior_citizen\'].astype(str)
    
    # Apply label encoding
    for col in config[\'categorical_cols\']:
        if col in label_encoders:
            df[col] = label_encoders[col].transform(df[col])
    
    # Apply scaling to numerical columns
    df[config[\'numerical_cols\']] = scaler.transform(df[config[\'numerical_cols\']])
    
    # Select features in correct order
    X = df[config[\'feature_cols\']].values
    
    return X

def get_risk_factors(data: dict, probability: float) -> List[str]:
    """Identify risk factors based on input data."""
    risk_factors = []
    
    if data.get(\'contract_type\') == "Month-to-month":
        risk_factors.append("Month-to-month contract (high churn risk)")
    if data.get(\'tenure\', 0) < 12:
        risk_factors.append("Short tenure (< 12 months)")
    if data.get(\'monthly_charges\', 0) > 70:
        risk_factors.append("High monthly charges")
    if data.get(\'tech_support\') == "No":
        risk_factors.append("No tech support")
    if data.get(\'online_security\') == "No":
        risk_factors.append("No online security")
    if data.get(\'num_tickets\', 0) > 3:
        risk_factors.append("Many support tickets")
    
    return risk_factors[:5]  # Top 5

def get_confidence(probability: float) -> str:
    """Get confidence level based on probability."""
    if probability < 0.3 or probability > 0.7:
        return "High"
    elif probability < 0.4 or probability > 0.6:
        return "Medium"
    else:
        return "Low"

# ========================================
# ENDPOINTS
# ========================================

@app.get("/", tags=["Root"])
async def root():
    """Root endpoint."""
    return {
        "message": "Churn Prediction API",
        "version": model_version,
        "docs": "/docs"
    }

@app.get("/health", response_model=HealthResponse, tags=["Health"])
async def health_check():
    """Health check endpoint."""
    return {
        "status": "healthy" if model is not None else "unhealthy",
        "model_loaded": model is not None,
        "model_version": model_version,
        "timestamp": datetime.utcnow().isoformat()
    }

@app.post("/predict", response_model=PredictionResponse, tags=["Prediction"])
async def predict(customer: CustomerData, customer_id: Optional[str] = None):
    """Make a single churn prediction."""
    try:
        # Convert to dict
        data = {
            "tenure": customer.tenure,
            "monthly_charges": customer.monthly_charges,
            "total_charges": customer.total_charges,
            "num_tickets": customer.num_tickets,
            "contract_type": customer.contract_type.value,
            "payment_method": customer.payment_method.value,
            "internet_service": customer.internet_service.value,
            "online_security": customer.online_security.value,
            "tech_support": customer.tech_support.value,
            "senior_citizen": customer.senior_citizen
        }
        
        # Preprocess
        X = preprocess_input(data)
        
        # Predict
        prediction = int(model.predict(X)[0])
        probability = float(model.predict_proba(X)[0, 1])
        
        # Get risk factors and confidence
        risk_factors = get_risk_factors(data, probability)
        confidence = get_confidence(probability)
        
        return {
            "customer_id": customer_id,
            "churn_prediction": prediction,
            "churn_probability": round(probability, 4),
            "confidence": confidence,
            "risk_factors": risk_factors
        }
        
    except Exception as e:
        logger.error(f"Prediction error: {e}")
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/predict/batch", response_model=BatchPredictionResponse, tags=["Prediction"])
async def predict_batch(request: BatchPredictionRequest):
    """Make batch churn predictions."""
    start_time = time.time()
    predictions = []
    
    for i, customer in enumerate(request.customers):
        try:
            result = await predict(customer, customer_id=f"batch_{i}")
            predictions.append(result)
        except Exception as e:
            logger.error(f"Batch prediction error for customer {i}: {e}")
            predictions.append({
                "customer_id": f"batch_{i}",
                "churn_prediction": -1,
                "churn_probability": 0.0,
                "confidence": "Error",
                "risk_factors": [str(e)]
            })
    
    processing_time = (time.time() - start_time) * 1000
    
    return {
        "predictions": predictions,
        "total_processed": len(predictions),
        "processing_time_ms": round(processing_time, 2)
    }

@app.get("/model/info", tags=["Model"])
async def model_info():
    """Get model information."""
    return {
        "model_type": type(model).__name__,
        "model_version": model_version,
        "features": config[\'feature_cols\'],
        "n_features": len(config[\'feature_cols\']),
        "numerical_features": config[\'numerical_cols\'],
        "categorical_features": config[\'categorical_cols\']
    }

if __name__ == "__main__":
    uvicorn.run("app:app", host="0.0.0.0", port=8000, reload=True)
'''

# Save FastAPI code
api_path = "./app.py"
with open(api_path, 'w') as f:
    f.write(fastapi_code)

print("\n" + "=" * 70)
print("FASTAPI APPLICATION")
print("=" * 70)
print(f"\n✅ FastAPI app saved to: {api_path}")
print("\nEndpoints:")
print("  GET  /              - Root endpoint")
print("  GET  /health        - Health check")
print("  POST /predict       - Single prediction")
print("  POST /predict/batch - Batch predictions")
print("  GET  /model/info    - Model information")
print("\nRun with: uvicorn app:app --reload")
print("Docs available at: http://localhost:8000/docs")


### 4.3 API Testing

Протестируем API локально с помощью Python requests.

In [ ]:
# Симуляция API запросов (без реального сервера)
# В реальности использовали бы requests.post()

print("\n" + "=" * 70)
print("API TESTING (SIMULATION)")
print("=" * 70)

# Test data
test_customer = {
    "tenure": 24,
    "monthly_charges": 65.5,
    "total_charges": 1500.0,
    "num_tickets": 2,
    "contract_type": "One year",
    "payment_method": "Credit card",
    "internet_service": "Fiber optic",
    "online_security": "Yes",
    "tech_support": "No",
    "senior_citizen": 0
}

print("\nTest Request:")
print(json.dumps(test_customer, indent=2))

# Simulate preprocessing and prediction
df_test = pd.DataFrame([test_customer])
df_test['senior_citizen'] = df_test['senior_citizen'].astype(str)

for col in categorical_cols:
    if col in label_encoders:
        df_test[col] = label_encoders[col].transform(df_test[col])

df_test[numerical_cols] = scaler.transform(df_test[numerical_cols])
X_test_api = df_test[feature_cols].values

prediction = int(best_model.predict(X_test_api)[0])
probability = float(best_model.predict_proba(X_test_api)[0, 1])

# Determine confidence
if probability < 0.3 or probability > 0.7:
    confidence = "High"
elif probability < 0.4 or probability > 0.6:
    confidence = "Medium"
else:
    confidence = "Low"

# Risk factors
risk_factors = []
if test_customer['contract_type'] == "Month-to-month":
    risk_factors.append("Month-to-month contract")
if test_customer['tenure'] < 12:
    risk_factors.append("Short tenure")
if test_customer['tech_support'] == "No":
    risk_factors.append("No tech support")

response = {
    "customer_id": "TEST-001",
    "churn_prediction": prediction,
    "churn_probability": round(probability, 4),
    "confidence": confidence,
    "risk_factors": risk_factors
}

print("\nSimulated Response:")
print(json.dumps(response, indent=2))

print("\n" + "=" * 70)
print("✅ API SIMULATION COMPLETE")
print("=" * 70)


---

## 🐳 Часть 5: Docker Containerization

### Почему Docker для ML?

**Проблемы без Docker:**
```
Data Scientist: "Работает на моей машине!" 🖥️
DevOps: "Не работает в production" 😤
Root cause: Python 3.8 vs 3.10, numpy 1.20 vs 1.24, etc.
```

**Docker решает:**
- ✅ **Reproducibility:** Одинаковое окружение везде
- ✅ **Isolation:** Нет конфликтов зависимостей
- ✅ **Portability:** Запускается на любом хосте с Docker
- ✅ **Scalability:** Легко масштабировать (Kubernetes)
- ✅ **CI/CD:** Automated builds и deployments

### Docker Architecture для ML:

```
┌─────────────────────────────────────────┐
│          Docker Container               │
├─────────────────────────────────────────┤
│  Application Layer                      │
│  ├── app.py (FastAPI)                   │
│  ├── models/ (serialized models)        │
│  └── requirements.txt                   │
├─────────────────────────────────────────┤
│  Python Runtime                         │
│  └── Python 3.9 + dependencies          │
├─────────────────────────────────────────┤
│  Base Image                             │
│  └── python:3.9-slim                    │
└─────────────────────────────────────────┘
```

---


In [ ]:
# Dockerfile
dockerfile_content = '''# Churn Prediction API - Production Dockerfile
# Multi-stage build for smaller image size

# ========================================
# Stage 1: Build dependencies
# ========================================
FROM python:3.9-slim as builder

# Set environment variables
ENV PYTHONDONTWRITEBYTECODE=1
ENV PYTHONUNBUFFERED=1

# Install build dependencies
RUN apt-get update && apt-get install -y --no-install-recommends \\
    build-essential \\
    && rm -rf /var/lib/apt/lists/*

# Create virtual environment
RUN python -m venv /opt/venv
ENV PATH="/opt/venv/bin:$PATH"

# Install Python dependencies
COPY requirements.txt .
RUN pip install --no-cache-dir --upgrade pip && \\
    pip install --no-cache-dir -r requirements.txt

# ========================================
# Stage 2: Production image
# ========================================
FROM python:3.9-slim as production

# Set environment variables
ENV PYTHONDONTWRITEBYTECODE=1
ENV PYTHONUNBUFFERED=1
ENV PORT=8000

# Create non-root user for security
RUN groupadd -r mluser && useradd -r -g mluser mluser

# Copy virtual environment from builder
COPY --from=builder /opt/venv /opt/venv
ENV PATH="/opt/venv/bin:$PATH"

# Set working directory
WORKDIR /app

# Copy application code
COPY app.py .
COPY models/ ./models/

# Change ownership to non-root user
RUN chown -R mluser:mluser /app

# Switch to non-root user
USER mluser

# Expose port
EXPOSE $PORT

# Health check
HEALTHCHECK --interval=30s --timeout=30s --start-period=5s --retries=3 \\
    CMD curl -f http://localhost:$PORT/health || exit 1

# Run application
CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8000"]
'''

# Save Dockerfile
with open('./Dockerfile', 'w') as f:
    f.write(dockerfile_content)

print("\n" + "=" * 70)
print("DOCKERFILE")
print("=" * 70)
print(f"\n✅ Dockerfile saved")
print("\nKey features:")
print("  - Multi-stage build (smaller image)")
print("  - Non-root user (security)")
print("  - Health check")
print("  - Optimized layer caching")


In [ ]:
# requirements.txt
requirements_content = '''# Core ML
numpy==1.24.3
pandas==2.0.3
scikit-learn==1.3.0
joblib==1.3.1

# API
fastapi==0.100.0
uvicorn[standard]==0.23.0
pydantic==2.0.3

# Monitoring
prometheus-client==0.17.1

# Utilities
python-multipart==0.0.6
python-json-logger==2.0.7
'''

with open('./requirements.txt', 'w') as f:
    f.write(requirements_content)

print("✅ requirements.txt saved")

# docker-compose.yml
docker_compose_content = '''version: '3.8'

services:
  churn-api:
    build:
      context: .
      dockerfile: Dockerfile
    container_name: churn-prediction-api
    ports:
      - "8000:8000"
    environment:
      - MODEL_VERSION=1.0.0
      - LOG_LEVEL=INFO
    volumes:
      - ./models:/app/models:ro
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:8000/health"]
      interval: 30s
      timeout: 10s
      retries: 3
      start_period: 10s
    restart: unless-stopped
    
  # Optional: Prometheus for monitoring
  prometheus:
    image: prom/prometheus:latest
    container_name: prometheus
    ports:
      - "9090:9090"
    volumes:
      - ./prometheus.yml:/etc/prometheus/prometheus.yml
    depends_on:
      - churn-api
      
  # Optional: Grafana for dashboards
  grafana:
    image: grafana/grafana:latest
    container_name: grafana
    ports:
      - "3000:3000"
    environment:
      - GF_SECURITY_ADMIN_PASSWORD=admin
    depends_on:
      - prometheus
'''

with open('./docker-compose.yml', 'w') as f:
    f.write(docker_compose_content)

print("✅ docker-compose.yml saved")

# .dockerignore
dockerignore_content = '''# Python
__pycache__/
*.py[cod]
*$py.class
*.so
.Python
venv/
env/
.venv/

# Jupyter
.ipynb_checkpoints/
*.ipynb

# Git
.git/
.gitignore

# IDE
.idea/
.vscode/

# Testing
tests/
*.test.py

# Documentation
docs/
*.md
README*

# MLflow
mlruns/
mlartifacts/

# Misc
.DS_Store
*.log
'''

with open('./.dockerignore', 'w') as f:
    f.write(dockerignore_content)

print("✅ .dockerignore saved")

print("\n" + "=" * 70)
print("DOCKER COMMANDS")
print("=" * 70)
print("\nBuild image:")
print("  docker build -t churn-api:latest .")
print("\nRun container:")
print("  docker run -p 8000:8000 churn-api:latest")
print("\nDocker Compose (with monitoring):")
print("  docker-compose up -d")
print("\nView logs:")
print("  docker logs churn-prediction-api")


---

## 📊 Часть 6: Monitoring & Drift Detection

### Почему Monitoring критически важен?

**ML системы деградируют со временем:**

1. **Data Drift:** Распределение входных данных меняется
   - Новые паттерны поведения клиентов
   - Сезонность
   - Изменение рынка

2. **Concept Drift:** Связь между features и target меняется
   - То, что раньше предсказывало churn, больше не работает
   - Модель устарела

3. **Model Degradation:** Accuracy падает со временем
   - Даже без drift, мир меняется
   - Нужно переобучение

### Типы метрик для мониторинга:

#### 1. **Input Data Metrics**
- Feature distributions (mean, std, min, max)
- Missing values
- Out-of-range values
- Feature correlations

#### 2. **Prediction Metrics**
- Prediction distribution
- Prediction confidence
- Prediction latency

#### 3. **Model Performance** (when labels available)
- Accuracy, Precision, Recall, F1
- ROC AUC
- Confusion matrix

#### 4. **Business Metrics**
- False positive cost
- Revenue impact
- Customer satisfaction

---


In [ ]:
print("=" * 70)
print("DATA DRIFT DETECTION")
print("=" * 70)

# Создадим синтетические "production" данные с drift
def create_drifted_data(n_samples: int = 1000, drift_level: float = 0.3) -> pd.DataFrame:
    """
    Create data with artificial drift.
    
    Args:
        n_samples: number of samples
        drift_level: intensity of drift (0=no drift, 1=full drift)
    """
    np.random.seed(123)  # Different seed for production data
    
    # Apply drift to numerical features
    tenure = np.random.randint(1, 72, n_samples)
    
    # Drift: monthly charges increased (например, инфляция)
    monthly_charges = np.random.uniform(
        20 + 15 * drift_level,  # higher minimum
        100 + 30 * drift_level,  # higher maximum
        n_samples
    )
    
    total_charges = tenure * monthly_charges + np.random.normal(0, 100, n_samples)
    total_charges = np.maximum(total_charges, 0)
    
    # Drift: more support tickets (ухудшилось качество услуг)
    num_tickets = np.random.poisson(2 + 2 * drift_level, n_samples)
    
    # Drift in categorical: more month-to-month contracts
    contract_probs = [
        0.5 + 0.2 * drift_level,  # more month-to-month
        0.3 - 0.1 * drift_level,  # less one year
        0.2 - 0.1 * drift_level   # less two year
    ]
    contract_probs = np.clip(contract_probs, 0.01, 0.99)
    contract_probs = contract_probs / sum(contract_probs)
    
    contract_type = np.random.choice(
        ['Month-to-month', 'One year', 'Two year'],
        n_samples, p=contract_probs
    )
    
    # Rest same as training
    payment_method = np.random.choice(
        ['Credit card', 'Bank transfer', 'Electronic check'],
        n_samples, p=[0.4, 0.3, 0.3]
    )
    internet_service = np.random.choice(
        ['DSL', 'Fiber optic', 'No'],
        n_samples, p=[0.4, 0.4, 0.2]
    )
    online_security = np.random.choice(['Yes', 'No'], n_samples, p=[0.4, 0.6])
    tech_support = np.random.choice(['Yes', 'No'], n_samples, p=[0.3, 0.7])
    senior_citizen = np.random.choice([0, 1], n_samples, p=[0.84, 0.16])
    
    df = pd.DataFrame({
        'tenure': tenure,
        'monthly_charges': monthly_charges,
        'total_charges': total_charges,
        'num_tickets': num_tickets,
        'contract_type': contract_type,
        'payment_method': payment_method,
        'internet_service': internet_service,
        'online_security': online_security,
        'tech_support': tech_support,
        'senior_citizen': senior_citizen,
    })
    
    return df

# Create production data with drift
production_data = create_drifted_data(1000, drift_level=0.4)

print("\n✅ Production data with drift created")
print(f"Shape: {production_data.shape}")


In [ ]:
from scipy import stats

def detect_numerical_drift(
    reference: pd.Series,
    current: pd.Series,
    threshold: float = 0.05
) -> Dict[str, Any]:
    """
    Detect drift in numerical feature using KS test.
    
    Kolmogorov-Smirnov test:
    - H0: distributions are the same
    - p-value < threshold → drift detected
    """
    statistic, p_value = stats.ks_2samp(reference, current)
    
    drift_detected = p_value < threshold
    
    return {
        'ks_statistic': statistic,
        'p_value': p_value,
        'drift_detected': drift_detected,
        'ref_mean': reference.mean(),
        'ref_std': reference.std(),
        'cur_mean': current.mean(),
        'cur_std': current.std(),
        'mean_shift': (current.mean() - reference.mean()) / reference.std()
    }

def detect_categorical_drift(
    reference: pd.Series,
    current: pd.Series,
    threshold: float = 0.05
) -> Dict[str, Any]:
    """
    Detect drift in categorical feature using Chi-squared test.
    """
    # Get value counts
    ref_counts = reference.value_counts(normalize=True)
    cur_counts = current.value_counts(normalize=True)
    
    # Align categories
    all_categories = set(ref_counts.index) | set(cur_counts.index)
    ref_aligned = np.array([ref_counts.get(cat, 0) for cat in all_categories])
    cur_aligned = np.array([cur_counts.get(cat, 0) for cat in all_categories])
    
    # Chi-squared test (need counts, not proportions)
    ref_counts_abs = ref_aligned * len(reference)
    cur_counts_abs = cur_aligned * len(current)
    
    # Add small value to avoid division by zero
    ref_counts_abs = ref_counts_abs + 1e-10
    
    statistic, p_value = stats.chisquare(cur_counts_abs, ref_counts_abs)
    
    drift_detected = p_value < threshold
    
    return {
        'chi2_statistic': statistic,
        'p_value': p_value,
        'drift_detected': drift_detected,
        'ref_distribution': ref_counts.to_dict(),
        'cur_distribution': cur_counts.to_dict()
    }

print("✅ Drift detection functions defined")


In [ ]:
# Run drift detection
print("\n" + "=" * 70)
print("DRIFT DETECTION RESULTS")
print("=" * 70)

# Reference data (training)
reference_data = df[df.columns.drop('churn')].copy()

drift_report = {}

# Numerical features
print("\n📊 Numerical Features:")
print("-" * 50)

for col in numerical_cols:
    result = detect_numerical_drift(
        reference_data[col],
        production_data[col]
    )
    drift_report[col] = result
    
    status = "⚠️ DRIFT" if result['drift_detected'] else "✅ OK"
    print(f"\n{col}:")
    print(f"  Status: {status}")
    print(f"  KS Statistic: {result['ks_statistic']:.4f}")
    print(f"  P-value: {result['p_value']:.4f}")
    print(f"  Mean shift: {result['mean_shift']:.2f} std")
    print(f"  Ref: mean={result['ref_mean']:.2f}, std={result['ref_std']:.2f}")
    print(f"  Cur: mean={result['cur_mean']:.2f}, std={result['cur_std']:.2f}")

# Categorical features
print("\n📊 Categorical Features:")
print("-" * 50)

for col in ['contract_type', 'payment_method', 'internet_service']:
    result = detect_categorical_drift(
        reference_data[col],
        production_data[col]
    )
    drift_report[col] = result
    
    status = "⚠️ DRIFT" if result['drift_detected'] else "✅ OK"
    print(f"\n{col}:")
    print(f"  Status: {status}")
    print(f"  Chi2 Statistic: {result['chi2_statistic']:.4f}")
    print(f"  P-value: {result['p_value']:.4f}")


In [ ]:
# Визуализация drift
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. monthly_charges distribution
axes[0, 0].hist(reference_data['monthly_charges'], bins=30, alpha=0.5, label='Reference', color='blue')
axes[0, 0].hist(production_data['monthly_charges'], bins=30, alpha=0.5, label='Production', color='red')
axes[0, 0].set_title('Monthly Charges Distribution', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Monthly Charges')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].legend()
if drift_report['monthly_charges']['drift_detected']:
    axes[0, 0].text(0.05, 0.95, '⚠️ DRIFT DETECTED', transform=axes[0, 0].transAxes,
                   fontsize=10, color='red', fontweight='bold', verticalalignment='top')

# 2. num_tickets distribution
axes[0, 1].hist(reference_data['num_tickets'], bins=15, alpha=0.5, label='Reference', color='blue')
axes[0, 1].hist(production_data['num_tickets'], bins=15, alpha=0.5, label='Production', color='red')
axes[0, 1].set_title('Number of Tickets Distribution', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Number of Tickets')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].legend()
if drift_report['num_tickets']['drift_detected']:
    axes[0, 1].text(0.05, 0.95, '⚠️ DRIFT DETECTED', transform=axes[0, 1].transAxes,
                   fontsize=10, color='red', fontweight='bold', verticalalignment='top')

# 3. contract_type comparison
contract_ref = reference_data['contract_type'].value_counts(normalize=True)
contract_prod = production_data['contract_type'].value_counts(normalize=True)

x = np.arange(len(contract_ref))
width = 0.35
axes[1, 0].bar(x - width/2, [contract_ref.get(c, 0) for c in contract_ref.index], width, label='Reference', alpha=0.8)
axes[1, 0].bar(x + width/2, [contract_prod.get(c, 0) for c in contract_ref.index], width, label='Production', alpha=0.8)
axes[1, 0].set_xticks(x)
axes[1, 0].set_xticklabels(contract_ref.index, rotation=45)
axes[1, 0].set_title('Contract Type Distribution', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Proportion')
axes[1, 0].legend()
if drift_report['contract_type']['drift_detected']:
    axes[1, 0].text(0.05, 0.95, '⚠️ DRIFT DETECTED', transform=axes[1, 0].transAxes,
                   fontsize=10, color='red', fontweight='bold', verticalalignment='top')

# 4. Summary
drift_summary = []
for feature, result in drift_report.items():
    if feature in numerical_cols:
        drift_summary.append({
            'Feature': feature,
            'Type': 'Numerical',
            'Drift': '⚠️ Yes' if result['drift_detected'] else '✅ No',
            'P-value': result['p_value']
        })
    else:
        drift_summary.append({
            'Feature': feature,
            'Type': 'Categorical',
            'Drift': '⚠️ Yes' if result['drift_detected'] else '✅ No',
            'P-value': result['p_value']
        })

summary_df = pd.DataFrame(drift_summary)
axes[1, 1].axis('off')
table = axes[1, 1].table(
    cellText=summary_df.values,
    colLabels=summary_df.columns,
    cellLoc='center',
    loc='center',
    colColours=['#f0f0f0'] * 4
)
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 1.5)
axes[1, 1].set_title('Drift Summary', fontsize=12, fontweight='bold', pad=20)

plt.suptitle('Data Drift Detection: Reference vs Production', fontsize=14, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

# Count drifted features
n_drifted = sum(1 for r in drift_report.values() if r['drift_detected'])
print(f"\n🔍 Summary: {n_drifted}/{len(drift_report)} features show significant drift")
if n_drifted > len(drift_report) // 2:
    print("\n⚠️ WARNING: Significant data drift detected!")
    print("   Consider retraining the model with recent data.")


### 6.1 Model Performance Monitoring

Когда появляются ground truth labels, мониторим performance.

In [ ]:
print("\n" + "=" * 70)
print("MODEL PERFORMANCE MONITORING")
print("=" * 70)

# Simulate predictions on production data
# Preprocess production data
prod_processed = production_data.copy()
prod_processed['senior_citizen'] = prod_processed['senior_citizen'].astype(str)

for col in categorical_cols:
    if col in label_encoders:
        prod_processed[col] = label_encoders[col].transform(prod_processed[col])

X_prod = prod_processed[feature_cols].copy()
X_prod[numerical_cols] = scaler.transform(X_prod[numerical_cols])

# Make predictions
prod_predictions = best_model.predict(X_prod)
prod_probabilities = best_model.predict_proba(X_prod)[:, 1]

# Simulate ground truth (with concept drift - higher actual churn rate)
# In reality, this comes from actual customer behavior
actual_churn_prob = (
    0.15 +  # higher baseline (was 0.1)
    (production_data['tenure'] < 12).astype(float) * 0.2 +  # stronger effect
    (production_data['monthly_charges'] > 70).astype(float) * 0.15 +
    (production_data['contract_type'] == 'Month-to-month').astype(float) * 0.25
)
actual_churn_prob = np.clip(actual_churn_prob, 0, 0.95)
prod_actual = (np.random.random(len(production_data)) < actual_churn_prob).astype(int)

# Calculate metrics
prod_accuracy = accuracy_score(prod_actual, prod_predictions)
prod_precision = precision_score(prod_actual, prod_predictions)
prod_recall = recall_score(prod_actual, prod_predictions)
prod_f1 = f1_score(prod_actual, prod_predictions)
prod_auc = roc_auc_score(prod_actual, prod_probabilities)

# Compare with training metrics
print("\n📊 Performance Comparison: Training vs Production")
print("-" * 50)

train_metrics = {
    'Accuracy': results['GradientBoosting']['accuracy'],
    'Precision': results['GradientBoosting']['precision'],
    'Recall': results['GradientBoosting']['recall'],
    'F1': results['GradientBoosting']['f1'],
    'ROC AUC': results['GradientBoosting']['roc_auc']
}

prod_metrics = {
    'Accuracy': prod_accuracy,
    'Precision': prod_precision,
    'Recall': prod_recall,
    'F1': prod_f1,
    'ROC AUC': prod_auc
}

comparison_data = []
for metric in train_metrics.keys():
    train_val = train_metrics[metric]
    prod_val = prod_metrics[metric]
    diff = prod_val - train_val
    pct_change = (diff / train_val) * 100
    
    status = "⚠️" if abs(pct_change) > 5 else "✅"
    
    comparison_data.append({
        'Metric': metric,
        'Training': f"{train_val:.4f}",
        'Production': f"{prod_val:.4f}",
        'Change': f"{pct_change:+.1f}%",
        'Status': status
    })
    
    print(f"{metric}:")
    print(f"  Training:   {train_val:.4f}")
    print(f"  Production: {prod_val:.4f}")
    print(f"  Change:     {pct_change:+.1f}% {status}")

comparison_df = pd.DataFrame(comparison_data)
print("\n" + comparison_df.to_string(index=False))

# Alert if significant degradation
if any(abs(float(row['Change'].replace('%', ''))) > 10 for row in comparison_data):
    print("\n" + "⚠️" * 20)
    print("\n🚨 ALERT: Significant model performance degradation detected!")
    print("   Actions:")
    print("   1. Investigate data drift")
    print("   2. Check for data quality issues")
    print("   3. Consider model retraining")
    print("   4. Review feature engineering")
    print("\n" + "⚠️" * 20)


---

## 🔄 Часть 7: CI/CD для ML

### Почему CI/CD для ML отличается от обычного?

**Обычный CI/CD:**
```
Code → Build → Test → Deploy
```

**ML CI/CD:**
```
Data → Feature Engineering → Train → Validate → Test → Package → Deploy → Monitor
```

**Дополнительные challenges:**
- ✅ **Data validation:** Проверка качества данных
- ✅ **Model validation:** Проверка качества модели
- ✅ **Feature validation:** Проверка feature distributions
- ✅ **Performance regression:** Модель не должна деградировать
- ✅ **A/B testing:** Сравнение с production моделью

### GitHub Actions для ML

**Workflow stages:**

1. **On Push/PR:**
   - Lint code (flake8, black)
   - Run unit tests
   - Run integration tests
   - Validate data schema

2. **On Model Training:**
   - Train model
   - Validate model metrics
   - Compare with baseline
   - Register model (if better)

3. **On Deploy:**
   - Build Docker image
   - Push to registry
   - Deploy to staging
   - Run smoke tests
   - Deploy to production

---


In [ ]:
# GitHub Actions Workflow
github_actions_workflow = '''name: ML Pipeline CI/CD

on:
  push:
    branches: [main, develop]
  pull_request:
    branches: [main]
  workflow_dispatch:  # Manual trigger

env:
  PYTHON_VERSION: "3.9"
  MODEL_NAME: "churn-predictor"

jobs:
  # ========================================
  # Stage 1: Code Quality
  # ========================================
  lint-and-test:
    name: Lint and Test
    runs-on: ubuntu-latest
    
    steps:
      - name: Checkout code
        uses: actions/checkout@v3
        
      - name: Set up Python
        uses: actions/setup-python@v4
        with:
          python-version: ${{ env.PYTHON_VERSION }}
          
      - name: Cache pip dependencies
        uses: actions/cache@v3
        with:
          path: ~/.cache/pip
          key: ${{ runner.os }}-pip-${{ hashFiles("requirements.txt") }}
          
      - name: Install dependencies
        run: |
          pip install --upgrade pip
          pip install -r requirements.txt
          pip install pytest pytest-cov flake8 black
          
      - name: Lint with flake8
        run: flake8 . --count --select=E9,F63,F7,F82 --show-source --statistics
        
      - name: Check formatting with black
        run: black --check .
        
      - name: Run unit tests
        run: pytest tests/unit/ -v --cov=src --cov-report=xml
        
      - name: Upload coverage
        uses: codecov/codecov-action@v3
        with:
          file: ./coverage.xml

  # ========================================
  # Stage 2: Data Validation
  # ========================================
  data-validation:
    name: Validate Data
    runs-on: ubuntu-latest
    needs: lint-and-test
    
    steps:
      - name: Checkout code
        uses: actions/checkout@v3
        
      - name: Set up Python
        uses: actions/setup-python@v4
        with:
          python-version: ${{ env.PYTHON_VERSION }}
          
      - name: Install dependencies
        run: pip install -r requirements.txt
        
      - name: Validate data schema
        run: python scripts/validate_data.py
        
      - name: Check for data drift
        run: python scripts/check_drift.py

  # ========================================
  # Stage 3: Model Training
  # ========================================
  train-model:
    name: Train Model
    runs-on: ubuntu-latest
    needs: data-validation
    if: github.event_name == "push" && github.ref == "refs/heads/main"
    
    steps:
      - name: Checkout code
        uses: actions/checkout@v3
        
      - name: Set up Python
        uses: actions/setup-python@v4
        with:
          python-version: ${{ env.PYTHON_VERSION }}
          
      - name: Install dependencies
        run: pip install -r requirements.txt
        
      - name: Train model
        run: python scripts/train.py
        env:
          MLFLOW_TRACKING_URI: ${{ secrets.MLFLOW_TRACKING_URI }}
          
      - name: Validate model performance
        run: python scripts/validate_model.py
        
      - name: Upload model artifact
        uses: actions/upload-artifact@v3
        with:
          name: model
          path: models/

  # ========================================
  # Stage 4: Build and Push Docker Image
  # ========================================
  build-docker:
    name: Build Docker Image
    runs-on: ubuntu-latest
    needs: train-model
    
    steps:
      - name: Checkout code
        uses: actions/checkout@v3
        
      - name: Download model artifact
        uses: actions/download-artifact@v3
        with:
          name: model
          path: models/
          
      - name: Set up Docker Buildx
        uses: docker/setup-buildx-action@v2
        
      - name: Login to Docker Hub
        uses: docker/login-action@v2
        with:
          username: ${{ secrets.DOCKER_USERNAME }}
          password: ${{ secrets.DOCKER_PASSWORD }}
          
      - name: Build and push
        uses: docker/build-push-action@v4
        with:
          context: .
          push: true
          tags: |
            ${{ secrets.DOCKER_USERNAME }}/${{ env.MODEL_NAME }}:latest
            ${{ secrets.DOCKER_USERNAME }}/${{ env.MODEL_NAME }}:${{ github.sha }}
          cache-from: type=gha
          cache-to: type=gha,mode=max

  # ========================================
  # Stage 5: Deploy to Staging
  # ========================================
  deploy-staging:
    name: Deploy to Staging
    runs-on: ubuntu-latest
    needs: build-docker
    environment: staging
    
    steps:
      - name: Deploy to staging
        run: |
          echo "Deploying to staging environment..."
          # kubectl set image deployment/$MODEL_NAME ...
          # or: aws ecs update-service ...
          
      - name: Run smoke tests
        run: |
          echo "Running smoke tests..."
          # curl -X POST https://staging.api.com/predict ...
          
      - name: Notify on success
        run: echo "Staging deployment successful!"

  # ========================================
  # Stage 6: Deploy to Production
  # ========================================
  deploy-production:
    name: Deploy to Production
    runs-on: ubuntu-latest
    needs: deploy-staging
    environment: production
    
    steps:
      - name: Deploy to production
        run: |
          echo "Deploying to production environment..."
          
      - name: Notify team
        run: echo "Production deployment complete!"
'''

# Save workflow
import os
os.makedirs('.github/workflows', exist_ok=True)
with open('.github/workflows/ml-pipeline.yml', 'w') as f:
    f.write(github_actions_workflow)

print("\n" + "=" * 70)
print("GITHUB ACTIONS CI/CD WORKFLOW")
print("=" * 70)
print("\n✅ Workflow saved to: .github/workflows/ml-pipeline.yml")
print("\nStages:")
print("  1. lint-and-test      - Code quality checks")
print("  2. data-validation    - Data schema and drift")
print("  3. train-model        - Model training with MLflow")
print("  4. build-docker       - Build and push Docker image")
print("  5. deploy-staging     - Deploy to staging + smoke tests")
print("  6. deploy-production  - Deploy to production")


---

## 📁 Часть 8: Data Version Control (DVC)

### Проблема версионирования данных

**Git не подходит для данных:**
- ❌ Large files (GB, TB)
- ❌ Binary files (не видно diff)
- ❌ Repository bloat

**DVC решает:**
- ✅ Git-like workflow для данных
- ✅ Remote storage (S3, GCS, Azure Blob)
- ✅ Reproducibility
- ✅ Pipeline management

### Как работает DVC:

```
┌─────────────┐     ┌─────────────┐
│   Git       │     │   DVC       │
│   (code)    │     │   (data)    │
├─────────────┤     ├─────────────┤
│ - app.py    │     │ - data.csv  │
│ - train.py  │     │ - model.pkl │
│ - data.dvc  │ ←─→ │             │
└─────────────┘     └─────────────┘
       │                   │
       ↓                   ↓
   GitHub/GitLab      S3/GCS/Azure
```

---


In [ ]:
print("\n" + "=" * 70)
print("DVC (DATA VERSION CONTROL)")
print("=" * 70)

# DVC commands reference
dvc_commands = '''
# ========================================
# DVC Quick Reference
# ========================================

# Initialize DVC in project
dvc init

# Add remote storage (e.g., S3)
dvc remote add -d myremote s3://mybucket/dvc-store

# Track data files
dvc add data/raw/customers.csv
dvc add models/churn_predictor.joblib

# This creates:
# - data/raw/customers.csv.dvc (metadata, tracked by Git)
# - .gitignore (добавляет customers.csv)

# Commit metadata to Git
git add data/raw/customers.csv.dvc data/raw/.gitignore
git commit -m "Add raw customer data"

# Push data to remote storage
dvc push

# Pull data from remote
dvc pull

# ========================================
# DVC Pipelines
# ========================================

# Define pipeline in dvc.yaml:
# stages:
#   preprocess:
#     cmd: python src/preprocess.py
#     deps:
#       - data/raw/customers.csv
#       - src/preprocess.py
#     outs:
#       - data/processed/customers_clean.csv
#
#   train:
#     cmd: python src/train.py
#     deps:
#       - data/processed/customers_clean.csv
#       - src/train.py
#     outs:
#       - models/churn_predictor.joblib
#     metrics:
#       - metrics/scores.json

# Run pipeline
dvc repro

# Show pipeline DAG
dvc dag

# Compare metrics across experiments
dvc metrics show
dvc metrics diff

# ========================================
# Version switching
# ========================================

# Go back to specific version
git checkout v1.0
dvc checkout

# Compare data versions
dvc diff HEAD~1
'''

print(dvc_commands)

# Create dvc.yaml example
dvc_yaml = '''stages:
  preprocess:
    cmd: python src/preprocess.py
    deps:
      - data/raw/customers.csv
      - src/preprocess.py
    params:
      - preprocess.test_size
      - preprocess.random_state
    outs:
      - data/processed/X_train.csv
      - data/processed/X_test.csv
      - data/processed/y_train.csv
      - data/processed/y_test.csv

  train:
    cmd: python src/train.py
    deps:
      - data/processed/X_train.csv
      - data/processed/y_train.csv
      - src/train.py
    params:
      - train.n_estimators
      - train.max_depth
      - train.learning_rate
    outs:
      - models/churn_predictor.joblib
      - models/scaler.joblib
    metrics:
      - metrics/train_metrics.json:
          cache: false

  evaluate:
    cmd: python src/evaluate.py
    deps:
      - data/processed/X_test.csv
      - data/processed/y_test.csv
      - models/churn_predictor.joblib
      - src/evaluate.py
    metrics:
      - metrics/eval_metrics.json:
          cache: false
    plots:
      - plots/confusion_matrix.png
      - plots/roc_curve.png
'''

with open('./dvc.yaml', 'w') as f:
    f.write(dvc_yaml)

print("\n✅ dvc.yaml saved")
print("\nDVC Pipeline Stages:")
print("  1. preprocess → Train/test split, scaling")
print("  2. train      → Model training")
print("  3. evaluate   → Model evaluation, plots")


---

## 📋 Часть 9: Summary & Best Practices

### Production ML Checklist

#### ✅ **Experiment Tracking**
- [ ] MLflow/W&B setup для logging
- [ ] Parameters, metrics, artifacts logged
- [ ] Model Registry configured
- [ ] Experiment comparison UI

#### ✅ **Model Serialization**
- [ ] Model saved (joblib/pickle/ONNX)
- [ ] Preprocessing artifacts saved (scaler, encoders)
- [ ] Config file with feature columns
- [ ] Version metadata

#### ✅ **API Development**
- [ ] FastAPI/Flask application
- [ ] Pydantic schemas for validation
- [ ] Health check endpoint
- [ ] Error handling
- [ ] API documentation (Swagger)
- [ ] Rate limiting
- [ ] Authentication

#### ✅ **Containerization**
- [ ] Dockerfile (multi-stage)
- [ ] Non-root user
- [ ] Health check
- [ ] docker-compose.yml
- [ ] .dockerignore

#### ✅ **Monitoring**
- [ ] Data drift detection
- [ ] Model performance monitoring
- [ ] Prediction distribution tracking
- [ ] Latency monitoring
- [ ] Alerting setup
- [ ] Dashboard (Grafana)

#### ✅ **CI/CD**
- [ ] GitHub Actions workflow
- [ ] Unit tests
- [ ] Integration tests
- [ ] Data validation
- [ ] Model validation
- [ ] Automated deployment

#### ✅ **Data Versioning**
- [ ] DVC initialized
- [ ] Remote storage configured
- [ ] Data tracked
- [ ] Pipeline defined

---

### 🎯 Key Takeaways

#### 1. **Model в Jupyter - это 10% работы**
- Остальные 90% - это MLOps
- Deployment, monitoring, retraining
- Это отличает Data Scientist от ML Engineer

#### 2. **Reproducibility is King**
- MLflow для experiments
- DVC для данных
- Docker для environment
- Всё должно быть воспроизводимо!

#### 3. **Monitoring предотвращает катастрофы**
- ML модели деградируют со временем
- Data drift, concept drift, model drift
- Мониторинг → раннее обнаружение → быстрая реакция

#### 4. **Automation снижает риски**
- Manual deployments → ошибки
- CI/CD → consistent, reliable deployments
- Automated testing → catches bugs early

#### 5. **Start Simple, Iterate**
- Не нужен Kubernetes с первого дня
- Начните с Docker + простой CI/CD
- Добавляйте complexity по мере необходимости

---

### 🔗 Связь с предыдущими фазами

**Phase 7 использует все предыдущие знания:**

- **Phase 1-3:** Модели для deployment (RF, XGBoost, etc.)
- **Phase 4:** Transformers для сложных задач
- **Phase 5:** Anomaly detection для monitoring
- **Phase 6:** SHAP для explainability в production

**Phase 7 завершает цикл:**

```
Research → Development → Production → Monitoring → Research...
```

---

### 📚 Tools Reference

| Category | Tool | Purpose |
|----------|------|---------||
| **Experiment Tracking** | MLflow | Parameters, metrics, models |
| | Weights & Biases | Similar + collaboration |
| **Model Serving** | FastAPI | REST API |
| | BentoML | ML-focused serving |
| | Seldon/KServe | Kubernetes-native |
| **Containerization** | Docker | Environment isolation |
| | Kubernetes | Orchestration |
| **Data Versioning** | DVC | Git for data |
| | Delta Lake | ACID transactions |
| **Monitoring** | Evidently | ML monitoring |
| | Prometheus/Grafana | Metrics + dashboards |
| **CI/CD** | GitHub Actions | Automation |
| | GitLab CI | Alternative |
| **Feature Store** | Feast | Feature management |

---

## 🎉 Phase 7 Complete!

**Вы теперь знаете:**

- ✅ MLflow experiment tracking и Model Registry
- ✅ Model serialization (pickle, joblib)
- ✅ FastAPI для ML API с Pydantic validation
- ✅ Docker containerization (multi-stage builds)
- ✅ Data drift detection (KS test, Chi-squared)
- ✅ Model performance monitoring
- ✅ CI/CD с GitHub Actions
- ✅ DVC для data versioning

---

## 🏆 ML Roadmap Complete!

**Весь путь от начала до production:**

1. ✅ **Phase 1:** Classification Basics (Titanic)
2. ✅ **Phase 2:** Deep Learning (MNIST, PyTorch)
3. ✅ **Phase 3:** Advanced Ensembles (Stacking, Blending)
4. ✅ **Phase 4:** Transformers (Self-Attention, TFT)
5. ✅ **Phase 5:** Anomaly Detection (Isolation Forest, Autoencoders)
6. ✅ **Phase 6:** Explainable AI (SHAP, LIME, Fairness)
7. ✅ **Phase 7:** Production MLOps (FastAPI, Docker, CI/CD)

**Теперь вы готовы:**

- 🎯 Решать real-world ML задачи
- 🎯 Выбирать правильные модели и методы
- 🎯 Деплоить модели в production
- 🎯 Мониторить и поддерживать ML системы
- 🎯 Объяснять predictions и обеспечивать fairness

---

**🚀 You're now a Production-Ready ML Practitioner!**
